## Author : Yekta Amirkhalili 

---
***

#### Date of Last Edit : 
> April 27, 2023 



#### File Information
----------------------
1. **Version**: 07

***----------------------***
##### Edits:

What is the difference from previous version? 

#### 1
- fixed data types 
- manually added positions (why? in Notion)

#### 2
- added frequency and weighted frequency to dictionary 
- finished writing all functions
- added documentation (on Notion)
- got rid of the "results" 

***-----------------------***


## Automation: Weights are all the same?

This here is my process of checking whether or not all the weight columns work similar to each other. I will basically write a Python script that automates what I was doing in Excel.

Steps: 

0. Clean dataset and make it ready 
1. come up with rules for each column: “ex. total spending was Less than 6410 or not” and give 0 or 1 to each entry for the related column.  
2. count frequencies of 1’s (summation of all the rows for that column) 
3. Check: is the frequency number close enough to the frequency in the PUMPF file? 
4. weight column * rule column (”less than 6410”) = weighted frequency column 
5. summation of all the rows of weighted frequency column
6. Check: is the summation of weighted frequency close enough to the weighted frequency in the PUMPF file? 
    1. Yes: Then it’s working similarly 
    2. No: Then it’s different. 
7. Record all progress

### DATA ANALYSIS (short)

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
#read data 
og_df = pd.read_csv('wrk_ds_00.csv')
cp_df = og_df.copy() #will work on this 

In [3]:
og_df.head()

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,wrpg991,wrpg992,wrpg993,wrpg994,wrpg995,wrpg996,wrpg997,wrpg998,wrpg999,wrpg1000
0,100000,11,3,6,1,2,1,2,2,2,...,58.74490,0.00000,91.23504,37.73533,84.68159,43.01466,0.00000,56.83786,103.81114,0.00000
1,100001,59,1,6,2,2,1,2,2,2,...,7136.75133,0.00000,0.00000,5224.34222,9012.71353,2402.81888,4359.56914,2818.29886,0.00000,4805.17223
2,100002,35,1,2,2,2,1,1,2,3,...,0.00000,0.00000,3391.39522,3810.10298,4368.93902,8221.13902,0.00000,0.00000,3462.35492,7660.63229
3,100003,35,1,3,2,2,1,1,2,3,...,6254.81170,6895.04808,4913.14578,5521.61592,0.00000,0.00000,0.00000,15612.81091,0.00000,6483.30541
4,100004,24,1,6,2,2,2,2,2,3,...,1477.61679,0.00000,1217.01163,2722.26852,3028.14230,0.00000,2702.80816,4168.62901,0.00000,0.00000


In [4]:
og_df.columns.get_loc('wtpg')

170

In [5]:
#get some overall information on the dataset 
og_df_sub = og_df.iloc[:, :170]
og_df_sub.head()

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,dl_speed_more50,life_satisfaction,mentalhealth,visible_minority,disability_status,health,household_type,house_quantile_income,immigrant,household_size
0,100000,11,3,6,1,2,1,2,2,2,...,1,6,2,2,2,2,2,2,2,2
1,100001,59,1,6,2,2,1,2,2,2,...,6,9,2,2,2,2,2,4,2,2
2,100002,35,1,2,2,2,1,1,2,3,...,6,9,3,2,2,3,1,3,2,3
3,100003,35,1,3,2,2,1,1,2,3,...,6,9,2,2,2,1,2,2,2,3
4,100004,24,1,6,2,2,2,2,2,3,...,6,11,1,2,2,3,2,4,2,2


In [6]:
df_noWeights = og_df_sub.copy()

In [7]:
only_weights = og_df.iloc[:, 170:]
only_weights.head()

,wtpg,wrpg1,wrpg2,wrpg3,wrpg4,wrpg5,wrpg6,wrpg7,wrpg8,wrpg9,...,wrpg991,wrpg992,wrpg993,wrpg994,wrpg995,wrpg996,wrpg997,wrpg998,wrpg999,wrpg1000
0,45.17965,149.11093,0.00000,149.63066,0.00000,84.74563,0.00000,182.72995,50.43055,153.19554,...,58.74490,0.00000,91.23504,37.73533,84.68159,43.01466,0.00000,56.83786,103.81114,0.00000
1,2524.87054,4341.00760,7856.51990,4982.54580,2876.07653,2990.92407,9237.73885,2315.91625,2395.59320,5015.74080,...,7136.75133,0.00000,0.00000,5224.34222,9012.71353,2402.81888,4359.56914,2818.29886,0.00000,4805.17223
2,3720.74855,0.00000,3470.18526,10686.85924,0.00000,7389.08920,3743.67150,3538.84146,4176.14775,4105.81575,...,0.00000,0.00000,3391.39522,3810.10298,4368.93902,8221.13902,0.00000,0.00000,3462.35492,7660.63229
3,5802.87880,6201.46077,0.00000,0.00000,5982.75992,6253.22554,12333.67688,5744.61028,10343.24866,5259.79450,...,6254.81170,6895.04808,4913.14578,5521.61592,0.00000,0.00000,0.00000,15612.81091,0.00000,6483.30541
4,1449.85202,1522.92124,1711.91466,1448.57857,0.00000,0.00000,2833.25017,1547.04205,1316.13353,2450.45243,...,1477.61679,0.00000,1217.01163,2722.26852,3028.14230,0.00000,2702.80816,4168.62901,0.00000,0.00000


In [8]:
non_categorical = ['spend_online_dig_music','spend_online_dig_video','spend_online_dig_books','spend_online_dig_podcast','spend_online_dig_news','spend_online_dig_giftcards','spend_online_dig_gambling','spend_online_dig_gaming','spend_online_dig_storage','spend_online_dig_courses',
                   'spend_online_dig_software','spend_online_dig_other','total_online_spending','spend_online_phy_total','spend_p2p_acc','spend_online_services','total_internet_spending']
 
df_noWeights_cat = df_noWeights.copy()
df_noWeights_cat = df_noWeights_cat.drop(columns = non_categorical)

df_noWeights_cat.head()
 

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,dl_speed_more50,life_satisfaction,mentalhealth,visible_minority,disability_status,health,household_type,house_quantile_income,immigrant,household_size
0,100000,11,3,6,1,2,1,2,2,2,...,1,6,2,2,2,2,2,2,2,2
1,100001,59,1,6,2,2,1,2,2,2,...,6,9,2,2,2,2,2,4,2,2
2,100002,35,1,2,2,2,1,1,2,3,...,6,9,3,2,2,3,1,3,2,3
3,100003,35,1,3,2,2,1,1,2,3,...,6,9,2,2,2,1,2,2,2,3
4,100004,24,1,6,2,2,2,2,2,3,...,6,11,1,2,2,3,2,4,2,2


In [9]:
df_noWeights_cat = df_noWeights_cat.astype(object)
df_noWeights_cont = df_noWeights.loc[:,non_categorical]

In [10]:
working_df = pd.concat([df_noWeights_cat, df_noWeights_cont], axis = 1)

In [11]:
working_df.head()

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,spend_online_dig_gaming,spend_online_dig_storage,spend_online_dig_courses,spend_online_dig_software,spend_online_dig_other,total_online_spending,spend_online_phy_total,spend_p2p_acc,spend_online_services,total_internet_spending
0,100000,11,3,6,1,2,1,2,2,2,...,0,0,0,0,0,150,900,999996,999999,1050
1,100001,59,1,6,2,2,1,2,2,2,...,0,0,0,0,0,310,150,999996,999999,460
2,100002,35,1,2,2,2,1,1,2,3,...,0,0,0,0,0,150,1500,999996,500,2150
3,100003,35,1,3,2,2,1,1,2,3,...,0,60,1000,300,0,1910,6000,999996,3000,10910
4,100004,24,1,6,2,2,2,2,2,3,...,0,0,0,0,0,0,999999,999996,999999,0


### DATA Explanations 

In here, I will add information about the dataset in the form of code. 

In [12]:
province_codes = {
    'NFL': 10,
    'PRNEDW': 11,
    'NScotia': 12,
    'NBrun': 13,
    'Qbc': 24,
    'ON': 35,
    'Mntb': 46,
    'Sask': 47,
    'Albt': 48,
    'BC': 59,
}

population_centre_codes = {
    'urban': 1,
    'rural': 2,
    'PRNEDW island': 3
}

age_codes = {
    '15-24': 1,
    '25-34': 2,
    '35-44': 3,
    '45-54': 4,
    '55-64': 5,
    '65+': 6
}

gender_codes = {
    'male': 1,
    'female': 2
}

aboriginal_codes = {
    'aboriginal': 1,
    'nonabo': 2
}

language_codes = {
    'ENG': 1,
    'FRN': 2,
    'ENG and FRN': 3,
    'NO ENG NO FRN': 4
}

employement_codes = {
    'employed': 1,
    'unemployed': 2
}

student_codes = {
    'student': 1, 
    'not student': 2
}

education_codes = {
    'HS or less': 1,
    'Some post-secondary': 2,
    'Uni degree': 3
}

mobile_data_size_codes = {
    'less than 1GB': 1,
    '1 to 6 GB': 2,
    '6 to 10 GB': 3,
    '10 to 20 GB': 4,
    '20 to 30 GB': 5,
    '30GB+': 6
}

last_use_internet_codes = {
    '3 months to a year ago': 1,
    'more than 1 year ago': 2,
    'never': 3
}

frequency_smartphone_use_codes = {
    'every 5 mins': 1,
    'every 15 mins': 2,
    'every 30 mins': 3,
    'every hour': 4,
    'a few times': 5,
    'only once or less': 6
}

friend_rel_codes = {
    'completely dissatisfied': 1,
    'somewhat dissatisfied': 2,
    'neutral': 3,
    'somewhat satisfied': 4,
    'completely satisfied': 5
}

family_rel_codes = {
    'completely dissatisfied': 1,
    'somewhat dissatisfied': 2,
    'neutral': 3,
    'somewhat satisfied': 4,
    'completely satisfied': 5
}

useNet_comm_friends_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

useNet_plan_withFriends_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

useNet_comm_family_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

useNet_plan_withFamily_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

purchase_physicalGoods_frq_codes = {
    'weekly': 1,
    'few times a month': 2,
    'once a month': 3,
    'few times a year': 4
}

loyalty_subscription_codes = {
    'less than 200': 1,
    '200 to 500': 2,
    '500 to 1000': 3,
    '1000 to 5000': 4,
    '5000 or more': 5
}

onlineShop_security_concern_codes = {
    'extremely concerned': 1,
    'very concerned': 2,
    'somewhat concerned': 3,
    'a little bit concerned': 4,
    'not concerned at all': 5
}

socialMedia_security_concern_codes = {
    'extremely concerned': 1,
    'very concerned': 2,
    'somewhat concerned': 3,
    'a little bit concerned': 4,
    'not concerned at all': 5
}

workPlace_type_codes = {
    'fixed loc outside of home': 1,
    'non fixed loc outside of home': 2,
    'work at home': 3
}

online_earnings_codes = {
    '0': 1,
    '1 to 200': 2,
    '200 to 1000': 3,
    '1000 to 2000': 4,
    '2000 or more': 5
}

downloadSpeeds_lessthan50_codes = {
    'less than 50Mbps': 1,
    'More than 50Mbps': 2
}

downloadSpeeds_morethan50_codes = {
    'less than 5 Mbps': 1,
    '5 to 20': 2,
    '20 to 50': 3
}

# some covid related data points here... 

life_satisfaction_codes = {
    'extremely dissatisfied': 1,
    'very dissatisfied': 2,
    'dissatisfied': 3,
    'somewhat dissatisfied': 4,
    'neutral': 5,
    'somewhat satisfied': 6,
    'satisfied': 7,
    'very satisfied': 8,
    'extremely satisfied': 9,
    'Incredibly satisfied': 10
}

mentalHealth_codes = {
    'Excellent': 1,
    'Very good': 2,
    'Good': 3,
    'Fair': 4,
    'Poor': 5
}

physicalHealth_codes = {
    'Excellent': 1,
    'Very good': 2,
    'Good': 3,
    'Fair': 4,
    'Poor': 5
}

household_type_codes = {
    'kids under 18': 1,
    'no kids under 18': 2,
    'single person': 3,
    'other': 4
}

household_income_quantile_codes = {
    'Q1: lOE to 44,119': 1,
    'Q2: 44,120 to 75321': 2,
    'Q3: 75322 to 109431': 3,
    'Q4: 109432 to 162799': 4,
    'Q5: GOE to 162800': 5
}

household_size_codes = { #this one somehow aligns tho
    '1 person': 1,
    '2 people': 2,
    '3 people': 3,
    '4 people': 4,
    '5 or more people': 5
}

Continous Variables: 

EC_G010A, music streaming subs dollar amounts, 0 - 450 

EC_G010B, video streaming subs dollar amounts, 0 - 660 

EC_G010C, Ebooks subs dollar amounts, 0 - 360 

EC_G010D, Podcasts subs dollar amounts, 0 - 300

EC_G010E, Newspapers subs dollar amounts, 0 - 540 

EC_G010F, Giftcards purchased dollar amounts, 0 - 600

G, Gambling dollar amounts, 0 - 1200

H, Gaming dollar amount, 0 - 600

I, Data Storage dollar amount, 0 - 180

J, Courses dollar amount, 0 - 1500

K, Applications and software dollar amounts, 0 - 450 

EC_G010X, total spending online (digital goods), $ 0 - 6410 

==================

EC_G020A, physical goods total $ amount: $ 0 - 6000

==================

EC_G050B, P2P accommodation services total amount spent: $ 0 - 3900

EC_G060A, Online purchases services total amount spent: $ 0 - 3000

SUM_GTOT, Total spending online: $ 0 - 18310 




### STEP1. Information

This is a summary information from the pdf file in a List based on the pattern where every entry is a dictionary with the following information:  

```python

column_name = {
    'name': col_name,
    'position': 1, #integer value 
    'category': cat_type #type of category,
    'actual_sum_value': 100,
    'weighted_f_count': 150
}

```


In [13]:
PROVINCE =  {
        'name': 'province',
        'position': 7,
        'category': 'ON',
        'actual_sum_value': 3719,
        'weighted_f_count': 12388500
    }

LUC_RST =  {
        'name': 'pop_cent',
        'position': 9,
        'category': 'A',
        'actual_sum_value': 11550,
        'weighted_f_count': 26618418
    }

AGE_GRP =  {
        'name': 'age_grp',
        'position': 11,
        'category': 'A',
        'actual_sum_value': 100,
        'weighted_f_count': 150
    }

GENDER =  {
        'name': 'gender',
        'position': 13,
        'category': 'A',
        'actual_sum_value': 828,
        'weighted_f_count': 4547807
    }

G_ABM =  {
        'name': 'indigen',
        'position': 14,
        'category': 'A',
        'actual_sum_value': 516,
        'weighted_f_count': 762686
    }

LAN_G01 =  {
        'name': 'lang',
        'position': 15,
        'category': 'A',
        'actual_sum_value': 11080,
        'weighted_f_count': 20034107
    }

EMP =  {
        'name': 'emp',
        'position': 16,
        'category': 'A',
        'actual_sum_value': 8451,
        'weighted_f_count': 17311613
    }


ED_G10 =  {
        'name': 'edu',
        'position': 17,
        'category': 'A',
        'actual_sum_value': 1004,
        'weighted_f_count': 4151420
    }

G_EDU =  {
        'name': 'highestedu',
        'position': 18,
        'category': 'A',
        'actual_sum_value': 5746,
        'weighted_f_count': 10383246
    }

HOME_USE =  {
        'name': 'home_use',
        'position': 20,
        'category': 'A',
        'actual_sum_value': 15673,
        'weighted_f_count': 29784980
    }

AC_030A =  {
        'name': 'access_dataplan',
        'position': 28,
        'category': 'A',
        'actual_sum_value': 12231,
        'weighted_f_count': 25069005
    }

AC_040A = {
        'name': 'access_dataplanSize',
        'position': 29,
        'category': 'A',
        'actual_sum_value': 717,
        'weighted_f_count': 1320428
    }

AC_050A =  {
        'name': 'lastThreemonthsUsage',
        'position': 31,
        'category': 'A',
        'actual_sum_value': 15294,
        'weighted_f_count': 29182133
    }

INT_USE =  {
        'name': 'int_use',
        'position': 32,
        'category': 'A',
        'actual_sum_value': 15294,
        'weighted_f_count': 29182133
    }


DV_010A =  {
        'name': 'DVC_smartphone',
        'position': 49,
        'category': 'A',
        'actual_sum_value': 12377,
        'weighted_f_count': 25545194
    }

DV_010B =  {
        'name': 'DVC_laptop',
        'position': 50,
        'category': 'A',
        'actual_sum_value': 9689,
        'weighted_f_count': 20266135
    }

DV_010C =  {
        'name': 'DVC_tablet',
        'position': 51,
        'category': 'A',
        'actual_sum_value': 7649,
        'weighted_f_count': 14115127
    }

DV_010D =  {
        'name': 'DVC_pc',
        'position': 52,
        'category': 'A',
        'actual_sum_value': 6314,
        'weighted_f_count': 12378695
    }

DV_010F =  {
        'name': 'DVC_TV',
        'position': 54,
        'category': 'A',
        'actual_sum_value': 5183,
        'weighted_f_count': 11164255
    }

DV_010G =  {
        'name': 'DVC_wearables',
        'position': 55,
        'category': 'A',
        'actual_sum_value': 1981,
        'weighted_f_count': 4392039
    }

DV_010H =  {
        'name': 'DVC_vehicle',
        'position': 56,
        'category': 'A',
        'actual_sum_value': 1311,
        'weighted_f_count': 2610307
    }




In [14]:
SM_G010A =  {
        'name': 'have_smartphone',
        'position': 74,
        'category': 'A',
        'actual_sum_value': 13108,
        'weighted_f_count': 26554185
    }

SM_USE =  {
        'name': 'sm_use',
        'position': 75,
        'category': 'A',
        'actual_sum_value': 13108,
        'weighted_f_count': 26554185
    }

SM_020C =  {
        'name': 'noUse_sm_difficult',
        'position': 78,
        'category': 'A',
        'actual_sum_value': 239,
        'weighted_f_count': 292274
    }

SM_020D =  {
        'name': 'noUse_sm_security',
        'position': 79,
        'category': 'A',
        'actual_sum_value': 146,
        'weighted_f_count': 189745
    }

SM_030A =  {
        'name': 'sm_use_frqnc',
        'position': 82,
        'category': 'A',
        'actual_sum_value': 673,
        'weighted_f_count': 2064135
    }

SM_040A =  {
        'name': 'smUse_time_bed',
        'position': 84,
        'category': 'A',
        'actual_sum_value': 6892,
        'weighted_f_count': 16141759
    }

SM_040B =  {
        'name': 'smUse_time_wakeUp',
        'position': 85,
        'category': 'A',
        'actual_sum_value': 7422,
        'weighted_f_count': 16743320
    }

SM_040C =  {
        'name': 'smUse_time_TV',
        'position': 86,
        'category': 'A',
        'actual_sum_value': 6722,
        'weighted_f_count': 14792803
    }

SM_040D =  {
        'name': 'smUse_time_dinner',
        'position': 87,
        'category': 'A',
        'actual_sum_value': 1550,
        'weighted_f_count': 4108740
    }

#SKIPPING THE ACTIVITIES ... FOR NOW (APR25TH)

TS_010A =  {
        'name': 'rel_friends',
        'position': 137,
        'category': 'A',
        'actual_sum_value': 236,
        'weighted_f_count': 419797
    }

TS_010B =  {
        'name': 'rel_relatives',
        'position': 138,
        'category': 'A',
        'actual_sum_value': 260,
        'weighted_f_count': 472765
    }


TS_020B =  {
        'name': 'socLife_hangOut',
        'position': 141,
        'category': 'A',
        'actual_sum_value': 443,
        'weighted_f_count': 839556
    }

TS_020D =  {
        'name': 'socLife_hangOutwFam',
        'position': 145,
        'category': 'A',
        'actual_sum_value': 4102,
        'weighted_f_count': 7783345
    }

#SKIPPING GOVERNMENT STUFF


EC_080D = {
    'name': 'payMethod_credit',
    'position': 284,
    'category': 'A',
    'actual_sum_value': 2480,
    'weighted_f_count': 5958245
}

SP_020C = {
    'name': 'Use_2FA',
    'position': 359,
    'category': 'A',
    'actual_sum_value': 7791,
    'weighted_f_count': 16932123
}

SP_040F = {
    'name': 'cyber_incident_payment',
    'position': 381,
    'category': 'A',
    'actual_sum_value': 1053,
    'weighted_f_count': 2229516
}

OW_060A = {
    'name': 'WFH_hours',
    'position': 404,
    'category': 'A',
    'actual_sum_value': 1480,
    'weighted_f_count': 3076568
}

FD_030A = {
    'name': 'mentalhealth',
    'position': 461,
    'category': 'A',
    'actual_sum_value': 3659,
    'weighted_f_count': 6196066
}

In [15]:
#continous ones...

EC_G010A =  {
        'name': 'spend_online_dig_music',
        'position': 172,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010B =  {
        'name': 'spend_online_dig_video',
        'position': 178,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010C =  {
        'name': 'spend_online_dig_books',
        'position': 184,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010D =  {
        'name': 'spend_online_dig_podcast',
        'position': 190,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010E =  {
        'name': 'spend_online_dig_news',
        'position': 196,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010F =  {
        'name': 'spend_online_dig_giftcards',
        'position': 202,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010G =  {
        'name': 'spend_online_dig_gambling',
        'position': 208,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010H =  {
        'name': 'spend_online_dig_gaming',
        'position': 214,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010I =  {
        'name': 'spend_online_dig_storage',
        'position': 220,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010J =  {
        'name': 'spend_online_dig_courses',
        'position': 226,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010K =  {
        'name': 'spend_online_dig_software',
        'position': 232,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010L =  {
        'name': 'spend_online_dig_other',
        'position': 238,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G010X =  {
        'name': 'total_online_spending',
        'position': 244,
        'category': 'cont',
        'actual_sum_value': 14579,
        'weighted_f_count': 27843326
    }

EC_G020A = {
    'name': 'spend_online_phy_total',
    'position': 252,
    'category': 'cont',
    'actual_sum_value': 11908,
    'weighted_f_count': 24003578
}

EC_G050B = {
    'name': 'spend_p2p_acc',
    'position': 266,
    'category': 'cont',
    'actual_sum_value': 1310,
    'weighted_f_count': 2974719
}

EC_G060A = {
    'name': 'spend_online_services',
    'position': 272,
    'category': 'cont',
    'actual_sum_value': 7852,
    'weighted_f_count': 17250184
}

SUM_GTOT = {
    'name': 'total_internet_spending',
    'position': 321,
    'category': 'cont',
    'actual_sum_value': 14718,
    'weighted_f_count': 28042611
}




In [18]:
lookUpInfo = [PROVINCE,LUC_RST,AGE_GRP,GENDER, G_ABM,LAN_G01, EMP,ED_G10,G_EDU,HOME_USE,AC_030A,
              AC_040A, AC_050A,INT_USE,DV_010A,SM_G010A,SM_USE,SM_020C,SM_030A,SM_040A,TS_010A,TS_020D,EC_080D,
              SP_020C, SP_040F,OW_060A,FD_030A,EC_G010A,EC_G010B,EC_G010C,EC_G020A,EC_G050B,EC_G060A,SUM_GTOT]

In [100]:
#making the dataset that includes only these: 

columns_to_include = []

for i in range(len(lookUpInfo)):
    columns_to_include.insert(i, lookUpInfo[i]['name'])

columns_to_include[:5]

['province', 'pop_cent', 'age_grp', 'gender', 'indigen']

In [101]:
df_smaller_toWork = working_df.loc[:, columns_to_include]

In [102]:
df_smaller_toWork.head()

,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,home_use,...,cyber_incident_payment,WFH_hours,mentalhealth,spend_online_dig_music,spend_online_dig_video,spend_online_dig_books,spend_online_phy_total,spend_p2p_acc,spend_online_services,total_internet_spending
0,11,3,6,1,2,1,2,2,2,1,...,2,6,2,0,150,0,900,999996,999999,1050
1,59,1,6,2,2,1,2,2,2,1,...,2,6,2,0,140,50,150,999996,999999,460
2,35,1,2,2,2,1,1,2,3,1,...,2,2,3,0,0,0,1500,999996,500,2150
3,35,1,3,2,2,1,1,2,3,1,...,2,6,2,200,120,200,6000,999996,3000,10910
4,24,1,6,2,2,2,2,2,3,2,...,2,6,1,0,0,0,999999,999996,999999,0


### STEP2. GET INFORMATION 

Here, I'll write a function to gather information by looking up the position and fetching the weight column. 

In [103]:
def get_info(col_name):
    """
    Looks into the LookUp list: 
        - finds the position of column, 
        - finds the categorization method for column 
        - returns a dictionary with the following info:
            * dataframe with column itself, weight column corresponding to it 
            * categorization method 
            * position number 
    """
    
    for i in range(len(lookUpInfo)):
        if lookUpInfo[i]['name'] == col_name: 
            pos_ = lookUpInfo[i]['position']
            cat_ = lookUpInfo[i]['category']
        else:
            continue
    
    
    s1 = pd.Series(cp_df[col_name])
    s2 = pd.Series(cp_df['wrpg{}'.format(pos_)])
    df_ = pd.concat([s1, s2], axis = 1)
    
    ret_dict = {
        'data': df_,
        'pos_': pos_,
        'cat_': cat_
    }
    return ret_dict 
    

### STEP3. CATEGORIZATIONS. 

This step gives information about how to give 0's and 1's to the data based on the question that was asked (how to categorize the answers). Some of the columns (questions) have more than 2 choices of answers, this means I will have to add a column for each answer and do 0's and 1's. For example, with age it was 15-25 (01), 25-34 (02), ..., 65 and older (06), etc. To save time, I'll just pick 1 of these and check. So, with age it will be "15-25"--- it's either 15-25 or it isn't. Then I'll check all the 15-24's and make my decision. 

In [104]:
def replace_with_binary(l, value_to_check, tp):
    """
    This is just to avoid repeatedly writing a loop and if statement
    """
    f = [] 
    
    if tp == 'cont':
        for i in range(len(l)):
            if l[i] <= value_to_check:
                f.insert(i, 1)
            else:
                f.insert(i, 0)
    else:
        for i in range(len(l)):
            if l[i] == value_to_check: 
                f.insert(i, 1)
            else:
                f.insert(i, 0)
            
    
    return f

In [105]:
#it is still looking at it as an integer value! 
ar = np.array(df_smaller_toWork['province'])
lar = list(ar)
lar.count(35)

3719

In [106]:
#match doesn't work for some reason, will do if-else

def categorize(col_name, categ):
    """
    Knowing the category, make a decision on how to fill up a frequency column.
    Anything other than a "1" will be written as a case, 
    All others will go to the default case
    """
#     freq = [] 
    
    li = list(np.array(df_smaller_toWork[col_name]))
        
    if categ == "ON":
    #ontario gets 1, everywhere else gets 0 
    #freq = replace_with_binary(li, "35", freq) 
    #MIGHT be an issue: passing freq, replacing it. 
        freq = replace_with_binary(li, 35, 'categorical')
        
    elif categ == 'cont':
        if col_name == 'spend_online_dig_music':
            freq = replace_with_binary(li, 450, 'cont')
        elif col_name == 'spend_online_dig_video':
            freq = replace_with_binary(li, 660, 'cont')
        elif col_name == 'spend_online_dig_books':
            freq = replace_with_binary(li, 360, 'cont')
        elif col_name == 'spend_online_phy_total':
            freq = replace_with_binary(li, 6000, 'cont')
        elif col_name == 'spend_p2p_acc':
            freq = replace_with_binary(li, 3900, 'cont')
        elif col_name == 'spend_online_services':
            freq = replace_with_binary(li, 3000, 'cont')
        elif col_name == 'total_internet_spending':
            freq = replace_with_binary(li, 18310, 'cont')
        else:
            freq = replace_with_binary(li, 500, 'cont')
        
    else:            
        freq = replace_with_binary(li, 1, 'categorical')
        

    return freq

Putting it all together: 

```python
get_info 
```

function gives me this result: 

```python

ret_dict = {
    'data': df_, #has the actual column and the weights 
    'pos_': pos_, 
    'cat_': cat_
}
```

- Take the df_ and add to it the frequency column from calling ```categorize```

In [107]:
def put_it_all_together(col_name):
    """
    Put the results from the two functions together in 1 dataframe for each column/variable. 
    """
    df_part1 = get_info(col_name)['data']
#     position = get_info(col_name)['pos']
    categ = get_info(col_name)['cat_']

    category_decision = categorize(col_name, categ)
    frq_ser = pd.Series(category_decision)
    
    df_full = df_part1.copy()
    df_full['category_decision'] = frq_ser
    
    return df_full
    

In [108]:
def apply_weights(data, p):
    """
    Formula: 
    weighted_frequency = category column * weight column 
    """
    
    cat_col = data['category_decision']
    wei_col = data['wrpg{}'.format(p)]
    
    data['weighted_frqn'] = cat_col * wei_col 
    
    return data
                   
            

In [109]:
def check_difference(weight_frq_col, values_frq_col, col_name):
    """
    Check the weighted frequency count here vs on the pdf file 
    
    values_frq_col === category_decision 
    weight_frq_col === result of apply_weights 
    """
    
    #actual data sum of frequencies 
    frq_sum_actual = sum(values_frq_col)
    
    #weighted data sum of frequencies 
    frq_sum_weighted = sum(weight_frq_col)
    
    
    for i in range(len(lookUpInfo)):
        if lookUpInfo[i]['name'] == col_name: #find the dictionary for this variable
            actual_sum = lookUpInfo[i]['actual_sum_value']
            weighted_sum = lookUpInfo[i]['weighted_f_count']
        else:
            continue #do nothing 
    
    
    #checking and results 
    print(col_name)
    
    print("---------------*****-----------------")
    print('Actual Values Frequency Sum: ', actual_sum)
    print('My calculated Values Frq Sum: ', frq_sum_actual)
    
    print("==============================================================")
    
    print('Actual weighted frequency sum: ', weighted_sum)
    print('My calculated weighted frequency sum:', frq_sum_weighted)
    
    if abs(frq_sum_actual - actual_sum) < 10000:
        print('The actual values frequency count matches.')
    else:
        print('*******WARNING*********')
        print('The actual values frequency count don\'t match.')
    
    if abs(frq_sum_weighted - weighted_sum) < 10000:
        print('The actual weighted frequency count matches.')
    else:
        print('*******WARNING*********')
        print('The actual weighted frequency count don\'t match.')
    
    
    print("==============================================================")
    print("==============================================================")

    
    

#### TESTING the code 

I will now try to test my code using just three columns. 

- province > PROVINCE
- edu > ED_G10
- spend_online_dig_music > EC_G010A



In [110]:
df_smaller_toWork.head()

,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,home_use,...,cyber_incident_payment,WFH_hours,mentalhealth,spend_online_dig_music,spend_online_dig_video,spend_online_dig_books,spend_online_phy_total,spend_p2p_acc,spend_online_services,total_internet_spending
0,11,3,6,1,2,1,2,2,2,1,...,2,6,2,0,150,0,900,999996,999999,1050
1,59,1,6,2,2,1,2,2,2,1,...,2,6,2,0,140,50,150,999996,999999,460
2,35,1,2,2,2,1,1,2,3,1,...,2,2,3,0,0,0,1500,999996,500,2150
3,35,1,3,2,2,1,1,2,3,1,...,2,6,2,200,120,200,6000,999996,3000,10910
4,24,1,6,2,2,2,2,2,3,2,...,2,6,1,0,0,0,999999,999996,999999,0


In [111]:
def sanity_check(var):
    """
    Automates All of it. Just provide the name of the column, and this will give you the answer. 
    """
    
    ############################################
    #        STEP 1. Get information           #                        
    ############################################
    
    
    infoDict = get_info(var)
    df_ = infoDict['data']
    pos__ = infoDict['pos_']
    cat__ = infoDict['cat_']
    

    ############################################
    #        STEP 2. Binary column             #      
    ############################################
    

    binaryList = categorize(var, cat__)

    
    ############################################
    #       STEP 3. Category decisions         #          
    ############################################
    

    df_withBinary = put_it_all_together(var)
    
    
    ############################################
    #       STEP 4. Calculate Weights          #                  
    ############################################
    

    df_final = apply_weights(df_withBinary, pos__)
    
    
    ############################################
    #       STEP 5. Calculate Difference       #                  
    ############################################

    
    res = check_difference(df_final['weighted_frqn'], df_final['category_decision'], var)

    return res

In [112]:
sanity_check('province')

province
---------------*****-----------------
Actual Values Frequency Sum:  3719
My calculated Values Frq Sum:  3719
Actual weighted frequency sum:  12388500
My calculated weighted frequency sum: 12388280.025153004
The actual values frequency count matches.
The actual weighted frequency count matches.


In [113]:
sanity_check('spend_online_dig_video')

spend_online_dig_video
---------------*****-----------------
Actual Values Frequency Sum:  14579
My calculated Values Frq Sum:  14579
Actual weighted frequency sum:  27843326
My calculated weighted frequency sum: 27912757.474890128
The actual values frequency count matches.
*******WARNING*********
The actual weighted frequency count don't match.


In [114]:
sanity_check('spend_online_dig_music')

spend_online_dig_music
---------------*****-----------------
Actual Values Frequency Sum:  14579
My calculated Values Frq Sum:  14579
Actual weighted frequency sum:  27843326
My calculated weighted frequency sum: 27806079.632108055
The actual values frequency count matches.
*******WARNING*********
The actual weighted frequency count don't match.


In [115]:
ed = sanity_check('edu')
ed

edu
---------------*****-----------------
Actual Values Frequency Sum:  1004
My calculated Values Frq Sum:  1004
Actual weighted frequency sum:  4151420
My calculated weighted frequency sum: 4070253.1629397003
The actual values frequency count matches.
*******WARNING*********
The actual weighted frequency count don't match.


In [116]:
type(ed)

NoneType

## Checking it all 

In [120]:
cols_ = list(df_smaller_toWork.columns)

# cols_

In [121]:
for col in cols_:
    sanity_check(col)

province
---------------*****-----------------
Actual Values Frequency Sum:  3719
My calculated Values Frq Sum:  3719
Actual weighted frequency sum:  12388500
My calculated weighted frequency sum: 12388280.025153004
The actual values frequency count matches.
The actual weighted frequency count matches.
pop_cent
---------------*****-----------------
Actual Values Frequency Sum:  11550
My calculated Values Frq Sum:  11550
Actual weighted frequency sum:  26618418
My calculated weighted frequency sum: 26634371.33915162
The actual values frequency count matches.
*******WARNING*********
The actual weighted frequency count don't match.
age_grp
---------------*****-----------------
Actual Values Frequency Sum:  100
My calculated Values Frq Sum:  828
Actual weighted frequency sum:  150
My calculated weighted frequency sum: 4548326.412444
The actual values frequency count matches.
*******WARNING*********
The actual weighted frequency count don't match.
gender
---------------*****----------------